## 測試項目
1. 訓練集中的alarm聲
2. 使用pruned and quantized tflite model
4. export to cc file

In [1]:
import os
import sys
import datetime
import shutil
import numpy as np
import tensorflow as tf
# from tensorflow import keras
import zipfile
import wavio
from common import utils as U
np.set_printoptions(threshold=sys.maxsize)

2024-01-19 11:57:34.885529: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-19 11:57:35.480800: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 11:57:35.480911: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 11:57:35.578065: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 11:57:35.779569: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from datetime import datetime
import re

### Loading TFlite Model

In [3]:
tflite_model_path = "../../refine_codes/trained_models/current_best/acdnet_alarm_3rd_20240119093633_acc_97.7272720336914_190th_epoch.pt";
# Load quantized TFLite model
tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_path)

ValueError: Model provided has model identifier '

In [4]:
# model.summary()
input_details = tflite_interpreter_quant.get_input_details()
output_details = tflite_interpreter_quant.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

#resize the input and output
# tflite_interpreter_quant.resize_tensor_input(input_details[0]['index'], (2, 1, 30225, 1))
# tflite_interpreter_quant.resize_tensor_input(output_details[0]['index'], (2, 2))

# input_details = tflite_interpreter_quant.get_input_details()
# output_details = tflite_interpreter_quant.get_output_details()

# print("== Input details ==")
# print("name:", input_details[0]['name'])
# print("shape:", input_details[0]['shape'])
# print("type:", input_details[0]['dtype'])

# print("\n== Output details ==")
# print("name:", output_details[0]['name'])
# print("shape:", output_details[0]['shape'])
# print("type:", output_details[0]['dtype'])

tflite_interpreter_quant.allocate_tensors();

== Input details ==
name: x.3
shape: [    1     1 30225     1]
type: <class 'numpy.float32'>

== Output details ==
name: 185
shape: [1 2]
type: <class 'numpy.float32'>


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### loading dataset from npz format
1. ACDNet input length is 30225
2. sr is 44100 and 20000
3. need to convert 16K to 20000
### ACDNet Config Setting
#### Training Parameters
1. opt.batchSize = 64;
2. opt.weightDecay = 5e-4;
3. opt.momentum = 0.9;
4. opt.nEpochs = 2000;
5. opt.LR = 0.1;
6. opt.schedule = [0.3, 0.6, 0.9];
7. opt.warmup = 10; 
#### Basic Net Configuration
- nClasses = 50
- nFolds = 5
- splits = \[i for in range(1, nFolds + 1)\]
- sr = 20000
- inputLength = 30225
<br>ngth = 30225;
### How to convert 16K sound to 44.1K with python and sox

if using sox the command is as following: <br />
    sox old.wav -b 16 new.wav 
if using python you can do as following: <br />
    import soundfile
    
data, samplerate = soundfile.read('old.wav
    <br />)
soundfile.write('new.wav', data, samplerate, subtype='PCM_1
6')

In [5]:
def genDataTimeStr():
    return datetime.today().strftime('%Y-%m-%d %H:%M:%S').replace('-',"").replace(' ',"").replace(':',"");

In [6]:
def getFileList(srcDir,regex='.*\.wav'):
    results = os.listdir(srcDir)
    out_files = []
    cnt_files = 0
    for file in results:
        if os.path.isdir(os.path.join(srcDir, file)):
            out_files += getFileList(os.path.join(srcDir, file))
        elif re.match(regex, file,  re.I):  # file.startswith(startExtension) or file.endswith(".txt") or file.endswith(endExtension):
            out_files.append(os.path.join(srcDir, file))
            cnt_files = cnt_files + 1
    return out_files

## sound preprocessing functions

In [7]:
_inputLen = 30225
_nCrops = 2
def preprocess_setup():
    funcs = []
    funcs += [U.padding( _inputLen// 2),
              U.normalize(32768.0),
              U.multi_crop(_inputLen, _nCrops)]
              # U.single_crop(_inputLen)]
              # 

    return funcs

def preprocess_debug():
    debug_funcs = []
    debug_funcs += [U.padding( _inputLen// 2),
              U.normalize(32768.0),]
              # U.multi_crop(_inputLen, _nCrops)]
              # U.single_crop(_inputLen)]
              # 

    return debug_funcs

def preprocess(sound, funcs):
    for f in funcs:
        sound = f(sound)

    return sound;

In [8]:
_funcs = preprocess_setup()
# _debug_funcs = preprocess_debug()

### Read Test Wav File

In [9]:
# for i in p_wav_list:
#     sound = wavio.read(i).data.T[0]
#     start = sound.nonzero()[0].min();
#     end = sound.nonzero()[0].max();
#     sound = sound[start: end + 1]
#     sound_debug = np.array(preprocess(sound,_debug_funcs));
#     print(f"sound content after pad and normalize:\n{sound_debug}")
#     label = int(p_label);
#     print(f"current sound start:{start} to end:{end}");
#     label = int(p_label);
#     if len(sound)> 220500:
#         sound = sound[:220500]
#     sound = preprocess(sound, _funcs);
#     print(f"label is {label} and sound len is {len(sound)}");
#     sounds.append(sound);
#     labels.append(label);
    
# for j in n_wav_list:
#     sound = wavio.read(j).data.T[0]
#     start = sound.nonzero()[0].min();
#     end = sound.nonzero()[0].max();
#     sound = sound[start: end + 1];
#     sound_debug = np.array(preprocess(sound,_debug_funcs));
#     print(f"sound content after pad and normalize:\n{sound_debug}")
#     label = int(n_label);
#     print(f"current sound start:{start} to end:{end}");
    
#     sound = np.float32(preprocess(sound, _funcs));
#     print(f"label is {label} and sound len is {len(sound)}")
#     sounds.append(sound);
#     labels.append(label);

In [20]:
positive_test_wavs = "../../../ACDNet_Test/sounds_for_test_from_testset/test_data_p_52/"
negative_test_wavs = "../../../ACDNet_Test/sounds_for_test_from_testset/test_data_n_99/"

p_wav_list = getFileList(positive_test_wavs);
n_wav_list = getFileList(negative_test_wavs);

# print(f"total alarm sounds:{len(p_wav_list)} ");
# print(f"total other sounds:{len(n_wav_list)}")
p_label = 52;
n_label = 99;

sounds = [];
labels = [];
## sound = preprocess(sound, _funcs)
for i in p_wav_list:
    sound = wavio.read(i).data.T[0]
    start = sound.nonzero()[0].min();
    end = sound.nonzero()[0].max();
    sound = sound[start: end + 1]
    label = int(p_label);
    if len(sound)> 220500:
        sound = sound[:220500]
    sound = preprocess(sound, _funcs);
    sounds.append(sound);
    labels.append(label);
    
for j in n_wav_list:
    sound = wavio.read(j).data.T[0]
    start = sound.nonzero()[0].min();
    end = sound.nonzero()[0].max();
    sound = sound[start: end + 1];
    label = int(n_label);
    sound = np.float32(preprocess(sound, _funcs));
    sounds.append(sound);
    labels.append(label);

print(f"sounds length: {len(sounds)}");
print(f"labels length: {len(labels)}");

# sound = wavio.read(test_sound_file).data.T[0]
# start = sound.nonzero()[0].min()
# end = sound.nonzero()[0].max()
# sound = sound[start: end + 1]  # Remove silent sections
# label = 18 #int(os.path.splitext(test_sound_file)[0].split('-')[-1])

# if len(sound)> 220500:
#     sound = sound[:220500]

# ec50_sound1 =  wavio.read(ec50_18_sound).data.T[0]
# start_ec50 = ec50_sound1.nonzero()[0].min()
# end_ec50 = ec50_sound1.nonzero()[0].max()
# ec50_sound1 = ec50_sound1[start_ec50:end_ec50+1]
# ec50_18_label = 18

sounds length: 58
labels length: 58


In [21]:
# print(sounds)
# print(labels)

In [22]:
print(sounds[0][1].shape)
print(type(labels[0]))

(30225,)
<class 'int'>


In [23]:
# sound = np.expand_dims(sound, axis=1)
# sound = np.expand_dims(sound, axis=3)
# print(sound.shape)

In [24]:
test_len = len(sounds);
p_total_len = len(p_wav_list);
n_total_len = len(n_wav_list);
correct_num = 0;
p_correct_num = 0;
n_correct_num = 0;
n_correct_siren_num = 0;
n_correct_othersounds_num = 0;
for w in range(test_len): 
    s = sounds[w]
    l = labels[w]
    s_test = np.expand_dims(s[1], axis=0);
    s_test = np.expand_dims(s_test, axis=1);
    s_test = np.expand_dims(s_test, axis=3);
    s_test = np.float32(s_test);
    # print(s_test.shape)
    # print(f"the {w+1} item's shape:\n  {s.shape}");
    # Run inference
    # print(f"s type:{type(s)}, and s shape:{s.shape}")
    tflite_interpreter_quant.set_tensor(input_details[0]['index'], s_test);
    tflite_interpreter_quant.invoke()
    pred = tflite_interpreter_quant.get_tensor(output_details[0]['index'])
    # print(f"Prediction result shape:{pred.shape}\n");
    print(f"Prediction result: {pred}, and true label: {l}")
    if l == 52: #positive
        if pred[0][0] > pred[0][1]:
            correct_num += 1;
            p_correct_num += 1;
    if l == 99: #negative
        if pred[0][0] < pred[0][1]:
            correct_num += 1;
            n_correct_num += 1;
print(f"total sounds result is {100*(correct_num/test_len)}");
print(f"total alarm sounds result is {100*(p_correct_num/p_total_len)}");
print(f"total siren sound result is {100*(n_correct_num/n_total_len)}");

Prediction result: [[0.94298524 0.05701476]], and true label: 52
Prediction result: [[0.98266476 0.0173352 ]], and true label: 52
Prediction result: [[9.9938345e-01 6.1656453e-04]], and true label: 52
Prediction result: [[0.99280506 0.00719495]], and true label: 52
Prediction result: [[0.986761   0.01323901]], and true label: 52
Prediction result: [[0.7493076  0.25069234]], and true label: 52
Prediction result: [[0.25069234 0.7493076 ]], and true label: 52
Prediction result: [[0.9980293  0.00197074]], and true label: 52
Prediction result: [[0.8183962  0.18160376]], and true label: 52
Prediction result: [[9.9996960e-01 3.0377752e-05]], and true label: 52
Prediction result: [[9.999027e-01 9.722223e-05]], and true label: 52
Prediction result: [[9.999993e-01 7.046469e-07]], and true label: 52
Prediction result: [[0.9351648  0.06483512]], and true label: 52
Prediction result: [[0.07364443 0.92635554]], and true label: 52
Prediction result: [[0.12829535 0.87170464]], and true label: 52
Predi

In [31]:
# scores = model.predict(sound, batch_size=len(sound), verbose=0);
# print(type(scores))
# print(scores.shape)

# for res in scores:
#     max_value = res.max()
#     max_index = np.argmax(res)
#     print(f"max value:{max_value:.5f} and index is {max_index}")
#     print('\n'.join('{}: {:.5f}'.format(*k) for k in enumerate(res)))